In [1]:
 pip install opencv-python numpy

In [2]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import numpy as np
from datasets import load_dataset
from PIL import Image

In [6]:
# 1. Hugging Face에서 데이터셋 불러오기
ds = load_dataset("ethz/food101", split="train")

# 2. 첫 번째 이미지 가져오기
pil_img = ds[0]['image']  # PIL 이미지
img_rgb = np.array(pil_img)  # RGB
img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)  # OpenCV용 BGR 변환

# 3. HSV 변환
hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)

# 4. 빨간색 범위 지정 (두 개 범위)
lower_red1 = np.array([0, 120, 70])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([170, 120, 70])
upper_red2 = np.array([180, 255, 255])

# 5. 마스크 생성
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
mask = cv2.bitwise_or(mask1, mask2)

# 6. 빨간색만 추출
result = cv2.bitwise_and(img_bgr, img_bgr, mask=mask)

# 7. 이미지 출력 (OpenCV 창)
cv2.imshow('Original Image', img_bgr)
cv2.imshow('Red Filtered Image', result)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# 1. 크기 조정 (224x224)
img_resized = cv2.resize(img_bgr, (224, 224))

# 2. Grayscale 변환 + 정규화 (0~1)
gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
gray_normalized = gray.astype(np.float32) / 255.0

# 3. 노이즈 제거
blurred = cv2.GaussianBlur(gray_normalized, (5, 5), 0)

# 4. 데이터 증강
# 좌우 반전
flipped = cv2.flip(img_resized, 1)

# 15도 회전
(h, w) = img_resized.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, 15, 1.0)
rotated = cv2.warpAffine(img_resized, M, (w, h))

# 색상 변화 (Hue +20 정도)
hsv = cv2.cvtColor(img_resized, cv2.COLOR_BGR2HSV)
hsv[:, :, 0] = (hsv[:, :, 0] + 20) % 180
color_jittered = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

cv2.imshow("Original", img_resized)
cv2.imshow("Gray + Normalized", gray_normalized)
cv2.imshow("Blurred", blurred)
cv2.imshow("Flipped", flipped)
cv2.imshow("Rotated", rotated)
cv2.imshow("Color Jittered", color_jittered)
cv2.waitKey(0)
cv2.destroyAllWindows()